But du notebook : recuperer et organiser les données afin de créer une carte interactive


# 1.Data loading


The data can be downloaded from the bixi montréal website.  
 https://bixi.com/fr/donnees-ouvertes  
you can download them by doing : 
 ```
 wget https://sitewebbixi.s3.amazonaws.com/uploads/docs/2021-stations-87d92f.zip 
 ```


In [1]:
import pandas as pd 
from datetime import datetime

In [2]:
deplacement = pd.read_csv('data/2021_donnees_ouvertes.csv')
deplacement.head()

,start_date,emplacement_pk_start,end_date,emplacement_pk_end,duration_sec,is_member
0,2021-05-21 17:21:44.934,10,2021-05-21 18:07:58.699,10,2773,1
1,2021-06-12 14:45:02.959,10,2021-06-12 15:11:14.937,187,1571,0
2,2021-08-27 08:19:17.396,10,2021-08-27 08:29:04.477,506,587,1
3,2021-06-11 19:07:28.724,10,2021-06-11 19:50:30.214,10,2581,0
4,2021-05-25 07:19:29.669,10,2021-05-25 07:47:42.001,423,1692,1


In [3]:
deplacement = deplacement.drop(columns=["end_date","duration_sec","is_member"])
deplacement_unique = deplacement.groupby(["emplacement_pk_start","emplacement_pk_end"]).count()
deplacement_unique = deplacement_unique.reset_index()
deplacement_unique.head()

,emplacement_pk_start,emplacement_pk_end,start_date
0,10,10,684
1,10,13,33
2,10,38,10
3,10,47,11
4,10,65,3


We success to import the 5 millions deplacements of Montrealers in 2021. 
Now we can import the stations because we have only the *pk* and not the exact emplacement

In [4]:
stations = pd.read_csv('data/2021_stations.csv')
stations.loc[stations["latitude"]==-1.0, "latitude"]=45.49277
stations.loc[stations["longitude"]==-1.0, "longitude"]=-73.55641
stations.head()


,pk,name,latitude,longitude
0,10,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630
1,13,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959
2,17,Tolhurst / Fleury,45.544149,-73.667520
3,21,St-Christophe / Cherrier,45.520290,-73.568119
4,31,Lespérance / de Rouen,45.538839,-73.552779


As you can see, we correct an error found earlier for the coordinates of the Peel/Smith station

We now perform a merge to obtain the geographical locations of each path

In [7]:
start = deplacement_unique.merge(stations, left_on="emplacement_pk_start", right_on="pk")
start.rename({"latitude":"pickup_latitude", "longitude":"pickup_longitude", "start_date":"nb", "name":"pickup_name"},errors="raise",axis='columns',inplace=True)
start_end = start.merge(stations, left_on="emplacement_pk_end", right_on="pk")
start_end.rename({"latitude":"dropoff_latitude", "longitude":"dropoff_longitude"},errors="raise",axis='columns',inplace=True)
deplacement_final = start_end.drop(columns=["emplacement_pk_end", "emplacement_pk_start","pk_x","pk_y"])
deplacement_final.head()

,nb,pickup_name,pickup_latitude,pickup_longitude,name,dropoff_latitude,dropoff_longitude
0,684,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
1,46,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
2,2,St-Christophe / Cherrier,45.520290,-73.568119,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
3,10,Métro Vendôme (de Marlowe / de Maisonneuve),45.473868,-73.604538,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363
4,20,Métro Guy-Concordia (Guy / St-Catherine),45.495045,-73.577830,Métro Angrignon (Lamont / des Trinitaires),45.44691,-73.60363


In [8]:
deplacement_final["name"]=deplacement_final["pickup_name"]+deplacement_final["name"]
deplacement_final.set_index("name",inplace=True)
deplacement_final.drop("pickup_name",axis=1)
deplacement_final.head()

,nb,pickup_name,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
name,,,,,,
Métro Angrignon (Lamont / des Trinitaires)Métro Angrignon (Lamont / des Trinitaires),684,Métro Angrignon (Lamont / des Trinitaires),45.446910,-73.603630,45.44691,-73.60363
Métro de l'Église (Ross / de l'Église)Métro Angrignon (Lamont / des Trinitaires),46,Métro de l'Église (Ross / de l'Église),45.462726,-73.565959,45.44691,-73.60363
St-Christophe / CherrierMétro Angrignon (Lamont / des Trinitaires),2,St-Christophe / Cherrier,45.520290,-73.568119,45.44691,-73.60363
Métro Vendôme (de Marlowe / de Maisonneuve)Métro Angrignon (Lamont / des Trinitaires),10,Métro Vendôme (de Marlowe / de Maisonneuve),45.473868,-73.604538,45.44691,-73.60363
Métro Guy-Concordia (Guy / St-Catherine)Métro Angrignon (Lamont / des Trinitaires),20,Métro Guy-Concordia (Guy / St-Catherine),45.495045,-73.577830,45.44691,-73.60363


In [9]:
from pathlib import Path
filepath = Path('data/deplacement.csv')
deplacement_final.to_csv(filepath)

A file containing all the trips is now savedA file containing all the trips is now saved and can be used in Keglergl.